# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [3]:
import pandas as pd 
import numpy as np

df = pd.read_csv("C:\\Users\\David Santos\\Desktop\\Labs longos\\lab-deep-learning\\your-code\\tic-tac-toe.csv")

df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
df.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [5]:
df.describe()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626


In [23]:
data_types = df.dtypes
print(data_types)

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [33]:
import tensorflow as tf 
from tensorflow import keras 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
pip install tensorflow

     -------------------------------------- 276.5/276.5 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 19.5 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 440.8/440.8 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 9.8 MB/s eta 0:00:00
     --------------------------------------- 24.4/24.4 MB 19.8 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 19.7 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 18.9 MB/s eta 0:00:00
     -------------------------------------- 181.4/181.4 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 151.7/151.7 kB ? eta 0:00:00
Note: you may need to restart the kernel to use up

In [17]:
# Convert categorical columns to one-hot encoded form
X_train_encoded = pd.get_dummies(X_train, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])
X_test_encoded = pd.get_dummies(X_test, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])

# Now you can proceed with StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [20]:
non_numeric_cols_train = X_train_encoded.select_dtypes(exclude=[np.number])
non_numeric_cols_test = X_test_encoded.select_dtypes(exclude=[np.number])


Empty DataFrame
Columns: []
Index: [302, 467, 294, 548, 465, 109, 77, 754, 259, 82, 495, 314, 328, 362, 483, 408, 2, 101, 351, 436, 247, 208, 5, 54, 694, 835, 613, 868, 97, 628, 204, 333, 692, 918, 507, 468, 832, 25, 84, 10, 377, 318, 527, 497, 519, 453, 118, 346, 357, 29, 55, 481, 331, 778, 829, 196, 422, 81, 299, 933, 593, 309, 841, 239, 884, 211, 687, 684, 780, 227, 820, 307, 846, 456, 874, 327, 424, 937, 332, 7, 155, 516, 644, 611, 388, 745, 541, 767, 551, 893, 668, 501, 796, 634, 405, 741, 228, 319, 715, 212, ...]

[766 rows x 0 columns]


In [27]:
df['class'] = df['class'].astype(int)

# Separate the inputs and output
X = df[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']]
y = df['class']

# Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical columns to one-hot encoded form
X_train_encoded = pd.get_dummies(X_train, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])
X_test_encoded = pd.get_dummies(X_test, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])

# Normalize the input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [30]:
X_train_encoded = pd.get_dummies(X_train, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])
X_test_encoded = pd.get_dummies(X_test, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])

In [34]:
# Convert 'class' column to integer values (0 and 1)
df['class'] = df['class'].astype(int)

# Separate the inputs and output
X = df[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']]
y = df['class']

# Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical columns to one-hot encoded form
X_train_encoded = pd.get_dummies(X_train, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])
X_test_encoded = pd.get_dummies(X_test, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])

# Normalize the input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Create a Sequential model using TensorFlow's integrated Keras
model = tf.keras.Sequential()

# Add layers to your model
model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))  # Assuming 2 classes, adjust as needed

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the training data
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate your neural network model with the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Save your model
model.save('tic-tac-toe.model')

Epoch 1/10
20/20 [==============================] - 1s 9ms/step - loss: 0.7738 - accuracy: 0.5114 - val_loss: 0.6017 - val_accuracy: 0.6883
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6027 - accuracy: 0.6797 - val_loss: 0.5605 - val_accuracy: 0.7208
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5578 - accuracy: 0.7157 - val_loss: 0.5358 - val_accuracy: 0.7662
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5137 - accuracy: 0.7516 - val_loss: 0.5126 - val_accuracy: 0.7597
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4742 - accuracy: 0.7876 - val_loss: 0.4907 - val_accuracy: 0.7792
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4397 - accuracy: 0.8121 - val_loss: 0.4700 - val_accuracy: 0.7857
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4042 - accuracy: 0.8235 - val_loss: 0.4458 - val_accuracy: 0.8052
Epoch 8/10
20/20 [==

INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


In [29]:
for column in X_train.columns:
    unique_values = X_train[column].unique()
    print(f"Column: {column}, Unique Values: {unique_values}")

Column: TL, Unique Values: ['o' 'x' 'b']
Column: TM, Unique Values: ['x' 'b' 'o']
Column: TR, Unique Values: ['x' 'o' 'b']
Column: ML, Unique Values: ['o' 'b' 'x']
Column: MM, Unique Values: ['x' 'b' 'o']
Column: MR, Unique Values: ['o' 'b' 'x']
Column: BL, Unique Values: ['b' 'x' 'o']
Column: BM, Unique Values: ['x' 'o' 'b']
Column: BR, Unique Values: ['b' 'x' 'o']


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [36]:
model = tf.keras.models.load_model('tic-tac-toe.model')

In [37]:
num_samples = 5  
random_indices = np.random.choice(X_test_scaled.shape[0], num_samples, replace=False)
random_test_data = X_test_scaled[random_indices]

In [38]:
predictions = model.predict(random_test_data)

1/1 [==============================] - 0s 74ms/step


In [39]:
predicted_labels = (predictions[:, 1] > 0.5).astype(int)

In [40]:
for i in range(num_samples):
    print(f"Sample {i + 1}:")
    print(f"Test Data (Scaled): {random_test_data[i]}")
    print(f"True Label: {y_test.iloc[random_indices[i]]}")
    print(f"Predicted Label: {predicted_labels[i]}")
    print()

Sample 1:
Test Data (Scaled): [-0.49755101  1.32118118 -0.88162474  1.72904116 -0.75051003 -0.7979675
 -0.513835   -0.74202917  1.13728324 -0.57232537  1.36708224 -0.82451474
 -0.45420173  1.35925849 -0.95655632 -0.59644967 -0.70849191  1.2161268
 -0.53205458 -0.7293833   1.14638536  1.71119991 -0.71891007 -0.82451474
 -0.52599113 -0.73358886  1.14334112]
True Label: 1
Predicted Label: 1

Sample 2:
Test Data (Scaled): [ 2.00984419 -0.75689846 -0.88162474 -0.57835523 -0.75051003  1.25318386
 -0.513835    1.34765592 -0.87928843  1.74725787 -0.73148489 -0.82451474
 -0.45420173 -0.73569524  1.04541675  1.67658739 -0.70849191 -0.82228267
  1.87950642 -0.7293833  -0.87230702 -0.58438526 -0.71891007  1.21283459
 -0.52599113  1.36316138 -0.87462961]
True Label: 1
Predicted Label: 1

Sample 3:
Test Data (Scaled): [ 2.00984419 -0.75689846 -0.88162474 -0.57835523  1.33242723 -0.7979675
 -0.513835    1.34765592 -0.87928843 -0.57232537 -0.73148489  1.21283459
 -0.45420173 -0.73569524  1.04541675 -0

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [41]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))  
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [44]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust the learning rate as needed
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)  # Adjust the number of epochs

Epoch 1/20
20/20 [==============================] - 1s 7ms/step - loss: 0.6703 - accuracy: 0.5817 - val_loss: 0.6044 - val_accuracy: 0.7013
Epoch 2/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5862 - accuracy: 0.6863 - val_loss: 0.5747 - val_accuracy: 0.7403
Epoch 3/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5387 - accuracy: 0.7141 - val_loss: 0.5511 - val_accuracy: 0.7273
Epoch 4/20
20/20 [==============================] - 0s 2ms/step - loss: 0.4871 - accuracy: 0.7794 - val_loss: 0.5285 - val_accuracy: 0.7208
Epoch 5/20
20/20 [==============================] - 0s 2ms/step - loss: 0.4344 - accuracy: 0.8154 - val_loss: 0.5003 - val_accuracy: 0.7403
Epoch 6/20
20/20 [==============================] - 0s 2ms/step - loss: 0.3767 - accuracy: 0.8464 - val_loss: 0.4748 - val_accuracy: 0.7403
Epoch 7/20
20/20 [==============================] - 0s 2ms/step - loss: 0.3205 - accuracy: 0.8758 - val_loss: 0.4372 - val_accuracy: 0.7468
Epoch 8/20
20/20 [==

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# adjusting the number of epochs.